In [121]:
!pip install websocket-client
!pip install json
!pip install numpy
!pip install python-binance
!pip install pandas


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [122]:
##
import websocket, json, pprint, numpy, ta
import pandas as pd
import numpy as np
from websocket import WebSocketApp
import config
from binance.client import Client
from binance.enums import *

import time
import logging


## I/ Moyenne du  1 sec bid and ask price

In [123]:
# Stream the market data:

#Carnet ordre pour ask et bid 1s
#BTC-USDT spot:
BTC_USDT = "wss://stream.binance.com:9443/ws/btcusdt@depth@1000ms"
#ETH-USDT spot:
ETH_USDT = "wss://stream.binance.com:9443/ws/ethusdt@depth@1000ms"
#BTC-USD perp:
BTC_USD_p = "wss://fstream.binance.com/ws/btcusdt@depth"
#ETH-USD perp:
ETH_USD_p = "wss://fstream.binance.com/ws/ethusdt@depth"


### Choix des variables d'intérêt

In [124]:
#Variable de choix

#Symbole:
SOCKET= BTC_USDT

#Level of quantity:
target=1


### Définition des fonctions pour le calcul de la moyenne du ask et bid 

In [125]:

#Définition des fonctions:

#Les fonctions s'appliquerons aux données du carnet d'ordre que nous obtiendrons via le websocket et prendrons comme deuxième argument les "targets" [1,10,25]


def Average_ask(data,level_quantity):
    total_quantity_ask= 0  # Quantité accumulée
    total_cost_ask = 0      # Coût total accumulé
   

    #Pour ce faire, nous utiliserons le carnet d'ordre et nous l'utiliserons de sorte à calculer quel est le "prix moyen du panier" qui nous permet d'acheter (au ask) la quantité cible.
    #Itération sur le carnet ordre au ask:
    for price, quantity in data:
        #On commence par le cas où l'offre de ask est soit plus grande que notre target ou fait excéder notre stock. Ainsi on considère une partie de l'offre.
        if total_quantity_ask + quantity > level_quantity:
            #Quantité considérée
            remaining_quantity = level_quantity - total_quantity_ask
            #"Cout" que cela nous rajoute
            total_cost_ask += remaining_quantity * price
            #La quantité totale devient notre quantité cible. On a atteint notre target. Fin de l'iteration
            total_quantity_ask += remaining_quantity
            break
        #Sinon, on "paye" la totalité de l'offre ask.
        else:
            total_cost_ask += price * quantity
            total_quantity_ask += quantity
    
    avg_ask=total_cost_ask / total_quantity_ask if total_quantity_ask > 0 else None
    return avg_ask

#De la même façon nous décrivons la fonction de calcul du bid average:

def Average_bid(data,level_quantity):
    total_quantity_bid = 0  # Quantité accumulée
    total_cost_bid = 0      # Coût total accumulé
    for price, quantity in data:

        if total_quantity_bid + quantity > level_quantity:
            #Quantité considéré
            remaining_quantity = level_quantity - total_quantity_bid
            #"Cout" que cela nous rajoute
            total_cost_bid += remaining_quantity * price
            #La quantité totale devient notre quantité cible. On a atteint notre target. Fin de l'iteration
            total_quantity_bid += remaining_quantity
            break
        else:
            total_cost_bid += price * quantity
            total_quantity_bid += quantity
    
    avg_bid=total_cost_bid / total_quantity_bid if total_quantity_bid> 0 else None
    return avg_bid



### Stream websocket: dictionnaire contenant le ask et bid moyen pour la quantité souhaitée

In [126]:
#Configuration du websocket
def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global Dictionnaire_ask_bid
    print('received message')
    json_message = json.loads(message)


    #Création du dictionnaire contenant le bid, ask average.
    Dictionnaire_ask_bid={}

    #Transformation de 'E' en date.
    times=pd.to_datetime(json_message['E'],unit='ms')
    formatted_times = times.strftime('%Y-%m-%d %H:%M:%S')
    Dictionnaire_ask_bid['time']=formatted_times

    #Intrument:
    Dictionnaire_ask_bid['Instrument']=json_message['s']

    #Type de contrat:
    #Nous pouvons le faire simplement via l'Url:
    if "fstream" in SOCKET:
        Dictionnaire_ask_bid['type']="perpetual"
    else:
        Dictionnaire_ask_bid['type']="spot"



    #Average bid and ask:

    #Récupération des données du carnet d'ordre:

    #ask
    data_float_ask = [(float(ask[0]), float(ask[1])) for ask in json_message['a']]

    #bid
    data_float_bid = [(float(bid[0]), float(bid[1])) for bid in json_message['b']]
   

   #Calcul des averages à l'aide des fonctions definis plus haut:
    avg_ask=Average_ask(data_float_ask,target)
    avg_bid=Average_bid(data_float_bid,target)


    #Ajout des variables dans le dictionnaire 
    Dictionnaire_ask_bid['total']=target
    Dictionnaire_ask_bid['avg_bid']=avg_bid
    Dictionnaire_ask_bid['avg_ask']=avg_ask

    print(Dictionnaire_ask_bid)
ws = websocket.WebSocketApp(SOCKET,on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()


opened connection
received message
{'time': '2024-03-13 15:03:41', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 1, 'avg_bid': 72340.01, 'avg_ask': 72340.02}
received message
{'time': '2024-03-13 15:03:42', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 1, 'avg_bid': 72310.20954139999, 'avg_ask': 72320.11953379998}
received message
{'time': '2024-03-13 15:03:43', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 1, 'avg_bid': 72301.29, 'avg_ask': 72301.3}
received message
{'time': '2024-03-13 15:03:44', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 1, 'avg_bid': 72303.93, 'avg_ask': 72308.9573236}
received message
{'time': '2024-03-13 15:03:45', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 1, 'avg_bid': 72306.5, 'avg_ask': 72306.51}
received message
{'time': '2024-03-13 15:03:46', 'Instrument': 'BTCUSDT', 'type': 'spot', 'total': 1, 'avg_bid': 72312.174562, 'avg_ask': 72319.66331029999}


True

### Visualisation du résultat obtenu

In [127]:
# Présentation en vertical
for cle, valeur in Dictionnaire_ask_bid.items():
    print(f"{cle}: {valeur}")


time: 2024-03-13 15:03:46
Instrument: BTCUSDT
type: spot
total: 1
avg_bid: 72312.174562
avg_ask: 72319.66331029999


## II/ Création du rapport csv

In [128]:
# Stream the market data:
BTC_USDT_analytics = "wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1s/btcusdt@depth"
ETH_USDT_analytics = "wss://stream.binance.com:9443/stream?streams=ethusdt@kline_1s/ethusdt@depth"
BTC_USD_p_analytics= "wss://fstream.binance.com/stream?streams=btcusdt_perpetual@continuousKline_1m/btcusdt@depth"
ETH_USD_p_analytics= "wss://fstream.binance.com/stream?streams=ethusdt_perpetual@continuousKline_1m/ethusdt@depth"

### Choix de la variable d'intérêt

In [129]:
SOCKET_analytics=BTC_USDT_analytics

### Stream websocket: Création d'un dataframe sur la période de streaming

In [130]:

# Initialisation des listes pour stocker les données
times_l = []  # Liste pour les timestamps
number_trades_l = []
traded_volume_l=[]
quotes_volume_l = []
symbole_l = []
spread_1_l=[]
spread_10_l=[]
spread_25_l=[]
type_instrument_l=[]

#Quant à la variable "Average Volume", nous la construirons nous même en calculant à chaque période la moyenne de l'ensemble des volumes tradés depuis le début du stream:
average_volume_l=[]

# Initialisation du DataFrame
analytics = pd.DataFrame()


#Configuration du websocket

def on_open(ws):
    print('opened connection')

def on_close(ws):
    print('closed connection')

def on_message(ws, message):
    global analytics
    print('received message')
    json_message = json.loads(message)


    data=json_message['data']

    #Comme ici nous utilisons un websocket combiné c'est à dire avec deux types de message (candle et orderbook). Il convient de sélectionner le bon message pour extraire les informations que nous souhaitons.


    #Création des variables spreads grâce à l'orderbook:

    #Identification du message 'orderbook'
    if 'a' in data and 'b' in data :
    
        #ask
        data_float_ask = [(float(ask[0]), float(ask[1])) for ask in data['a']]

        #bid
        data_float_bid = [(float(bid[0]), float(bid[1])) for bid in data['b']]

        #Calcul des spreads
        spread_1=Average_ask(data_float_ask,1)-Average_bid(data_float_bid,1)
        spread_10=Average_ask(data_float_ask,10)-Average_bid(data_float_bid,10)
        spread_25=Average_ask(data_float_ask,25)-Average_bid(data_float_bid,25)
    
        #ajout à la liste
        spread_1_l.append(float(spread_1))
        spread_10_l.append(float(spread_10))
        spread_25_l.append(float(spread_25))


        #Type de contrat:
        #Nous pouvons le faire simplement via l'Url, comme précédemment.
        if "fstream" in SOCKET:
            type_instrument_l.append("perpetual")
        else:
            type_instrument_l.append("spot")

        #Symbole
        symbole = data['s']
        symbole_l.append(symbole)
    


    #Création des autres variables d'intérêt
        
    #Identification du message 'candle'
    if 'k' in data:

        #Identification du dictionaire candle
        candle = data['k']

        time = pd.to_datetime(candle['t'], unit='ms')  # Conversion du timestamp en datetime
        number_trades = candle['n']
        quotes_volume = candle['q']
      
        traded_volume= candle['v']

        #ajout à la liste
        times_l.append(time)
        traded_volume_l.append(float(traded_volume))
        number_trades_l.append(float(number_trades))
        quotes_volume_l.append(float(quotes_volume))
        

        #Cas du volume moyen
        average_volume=np.mean(traded_volume_l)
        average_volume_l.append(float(average_volume))

    # Création du dataframe. Ce dernier va s'actualiser à chaque seconde
        
    analytics = pd.DataFrame({
        'time': times_l,
        'symbole':symbole_l,
        'type':type_instrument_l,
        'number of trades': number_trades_l,
        'traded volume': traded_volume_l,
        'average volume': average_volume_l,
        'quotes volume': quotes_volume_l,
        'spread_1':spread_1_l,
        'spread_10':spread_10_l,
        'spread_25':spread_25_l
    }).set_index('time')  # Définir 'time' comme index


    print(analytics)
    

ws = websocket.WebSocketApp(SOCKET_analytics, on_open=on_open, on_close=on_close, on_message=on_message)
ws.run_forever()
    

opened connection
received message
received message
                     symbole  type  number of trades  traded volume  \
time                                                                  
2024-03-13 15:04:03  BTCUSDT  spot               8.0        0.09868   

                     average volume  quotes volume  spread_1  spread_10  \
time                                                                      
2024-03-13 15:04:03         0.09868    7137.205664      0.01  25.093764   

                       spread_25  
time                              
2024-03-13 15:04:03  2528.594443  
received message
received message
                     symbole  type  number of trades  traded volume  \
time                                                                  
2024-03-13 15:04:03  BTCUSDT  spot               8.0        0.09868   
2024-03-13 15:04:04  BTCUSDT  spot              46.0        0.23046   

                     average volume  quotes volume  spread_1  spread_10  \
time     

True

### Visualisation du résultat obtenu

In [131]:
analytics

,symbole,type,number of trades,traded volume,average volume,quotes volume,spread_1,spread_10,spread_25
time,,,,,,,,,
2024-03-13 15:04:03,BTCUSDT,spot,8.0,0.09868,0.098680,7137.205664,0.010000,25.093764,2528.594443
2024-03-13 15:04:04,BTCUSDT,spot,46.0,0.23046,0.164570,16669.771261,0.279831,14.585498,47.803185
2024-03-13 15:04:05,BTCUSDT,spot,5.0,0.02089,0.116677,1511.202558,0.010000,25.764653,4027.503747
2024-03-13 15:04:06,BTCUSDT,spot,9.0,0.33804,0.172017,24454.178515,0.010000,1376.668631,2567.820470
2024-03-13 15:04:07,BTCUSDT,spot,64.0,2.55392,0.648398,184763.983207,0.283768,14.128149,43.763840
2024-03-13 15:04:08,BTCUSDT,spot,18.0,0.43728,0.613212,31641.161229,0.010000,18.784167,89.416975
2024-03-13 15:04:09,BTCUSDT,spot,103.0,2.06609,0.820766,149527.714915,8.374533,27.778952,1006.737701
2024-03-13 15:04:10,BTCUSDT,spot,6.0,0.01470,0.720007,1064.029239,0.010000,14.429779,43.100972


### Transformation du dataframe en fichier csv

In [132]:
# Transformer le data frame de la fin du stream en csv.

#Parametrisation du fichier csv:
symbole=analytics['symbole'].unique()[0]
contract=analytics['type'].unique()[0]
filename = f'analytics_instrument_{symbole}_{contract}.csv'

#Conversion du data frame en fichier csv
analytics.to_csv(filename)

### Visualisation du rapport csv

In [133]:
#Visualisation du fichier csv
Analytics_csv = pd.read_csv(filename, index_col='time')
Analytics_csv

,symbole,type,number of trades,traded volume,average volume,quotes volume,spread_1,spread_10,spread_25
time,,,,,,,,,
2024-03-13 15:04:03,BTCUSDT,spot,8.0,0.09868,0.098680,7137.205664,0.010000,25.093764,2528.594443
2024-03-13 15:04:04,BTCUSDT,spot,46.0,0.23046,0.164570,16669.771261,0.279831,14.585498,47.803185
2024-03-13 15:04:05,BTCUSDT,spot,5.0,0.02089,0.116677,1511.202558,0.010000,25.764653,4027.503747
2024-03-13 15:04:06,BTCUSDT,spot,9.0,0.33804,0.172017,24454.178515,0.010000,1376.668631,2567.820470
2024-03-13 15:04:07,BTCUSDT,spot,64.0,2.55392,0.648398,184763.983207,0.283768,14.128149,43.763840
2024-03-13 15:04:08,BTCUSDT,spot,18.0,0.43728,0.613212,31641.161229,0.010000,18.784167,89.416975
2024-03-13 15:04:09,BTCUSDT,spot,103.0,2.06609,0.820766,149527.714915,8.374533,27.778952,1006.737701
2024-03-13 15:04:10,BTCUSDT,spot,6.0,0.01470,0.720007,1064.029239,0.010000,14.429779,43.100972
